In [1]:
!pip install datasets transformers
!apt install git-lfs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 5.4 MB/s 
     |████████████████████████████████| 4.4 MB 50.8 MB/s 
     |████████████████████████████████| 212 kB 60.2 MB/s 
     |████████████████████████████████| 101 kB 11.1 MB/s 
     |████████████████████████████████| 1.1 MB 58.0 MB/s 
     |████████████████████████████████| 140 kB 46.0 MB/s 
     |████████████████████████████████| 596 kB 60.2 MB/s 
     |████████████████████████████████| 127 kB 61.3 MB/s 
     |████████████████████████████████| 6.6 MB 41.7 MB/s 
     |████████████████████████████████| 94 kB 3.4 MB/s 
     |████████████████████████████████| 144 kB 75.2 MB/s 
     |████████████████████████████████| 271 kB 52.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing

In [2]:
!git config --global user.email "siddhantsaraf1@gmail.com"
!git config --global user.name "Siddhant Saraf"

In [3]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [4]:
from datasets import load_dataset

english_dataset = load_dataset("amazon_reviews_multi", "en")
english_dataset

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/200000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Dataset amazon_reviews_multi downloaded and prepared to /root/.cache/huggingface/datasets/amazon_reviews_multi/en/1.0.0/724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 200000
    })
    validation: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 5000
    })
})

In [7]:
print(english_dataset['train'].shape)
print(english_dataset['test'].shape)

(200000, 8)
(5000, 8)


In [17]:
import pandas as pd

In [8]:
def show_samples(dataset, num_samples=3, seed=42):
    sample = dataset["train"].shuffle(seed=seed).select(range(num_samples))
    for example in sample:
        print(f"\n'>> Title: {example['review_title']}'")
        print(f"'>> Review: {example['review_body']}'")


show_samples(english_dataset)


'>> Title: Worked in front position, not rear'
'>> Review: 3 stars because these are not rear brakes as stated in the item description. At least the mount adapter only worked on the front fork of the bike that I got it for.'

'>> Title: meh'
'>> Review: Does it’s job and it’s gorgeous but mine is falling apart, I had to basically put it together again with hot glue'

'>> Title: Can't beat these for the money'
'>> Review: Bought this for handling miscellaneous aircraft parts and hanger "stuff" that I needed to organize; it really fit the bill. The unit arrived quickly, was well packaged and arrived intact (always a good sign). There are five wall mounts-- three on the top and two on the bottom. I wanted to mount it on the wall, so all I had to do was to remove the top two layers of plastic drawers, as well as the bottom corner drawers, place it when I wanted and mark it; I then used some of the new plastic screw in wall anchors (the 50 pound variety) and it easily mounted to the wall. 

In [9]:
english_dataset.set_format("pandas")
english_df = english_dataset["train"][:]
# Show counts for top 20 products
english_df["product_category"].value_counts()[:20]

home                      17679
apparel                   15951
wireless                  15717
other                     13418
beauty                    12091
drugstore                 11730
kitchen                   10382
toy                        8745
sports                     8277
automotive                 7506
lawn_and_garden            7327
home_improvement           7136
pet_products               7082
digital_ebook_purchase     6749
pc                         6401
electronics                6186
office_product             5521
shoes                      5197
grocery                    4730
book                       3756
Name: product_category, dtype: int64

In [13]:
def filter_home(example):
    return (
        example["product_category"] == "home"
    )

In [11]:
english_dataset.reset_format()

In [22]:
english_home = english_dataset.filter(filter_home)
show_samples(english_home)



'>> Title: Poor customer service purchase from a different company'
'>> Review: Poor customer service received broken just given the runaround poor value find another company'

'>> Title: Relief At Last!'
'>> Review: Extremely pleased. My desktop fan just didn't put out enough to keep me cool in an extremely hot office. I'm very happy this with fan. More bang for the buck, doesn't take up much space, and more importantly, keeps me cool!'

'>> Title: Really disappointed..'
'>> Review: Really disappointed... The elephant print looks nothing like the photo... It's vertically distorted and gets cut off at the sides.'


In [53]:
english_home['validation']

Dataset({
    features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
    num_rows: 241
})

In [56]:
# english_home = english_home.filter(lambda x: len(x["review_title"].split()) > 2)
english_home['train']['review_title'][0]

'The product is junk.'

In [135]:
from transformers import  TFGPT2LMHeadModel, GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = TFGPT2LMHeadModel.from_pretrained('gpt2', pad_token_id=tokenizer.eos_token_id)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [106]:
review_title = english_home['train']['review_title']
review_title[:5]

['The product is junk.',
 'Not reliable ☹️',
 'stems were broken',
 'This product is very easy to use',
 'Not happy with customer service.']

In [145]:
input_ids = tokenizer.encode(review_title[0:5], return_tensors='tf', max_length=50, truncation=True) # remove [0] to run for the entire dataset
# input_ids = tokenizer.encode(["my name is siddhant",'i am learning to code'], return_tensors = 'tf')


In [149]:
tokenizer.decode(input_ids[0])

'<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>'

In [151]:
output = model.generate(input_ids, max_length = 100, num_beams=5, no_repeat_ngram_size=2, early_stopping=True)
output

<tf.Tensor: shape=(1, 100), dtype=int32, numpy=
array([[50256, 50256, 50256, 50256, 50256,  1212,   318,   257, 10484,
        14687,    13, 17393,   743,   407,   307,   287,   663,  2457,
         1296,    13,   198,   198, 29428, 32021,    25,   770,   318,
        20265,  2735, 28265,  7996,  2197,    13,  2398,    11,   383,
         1810,   290, 12689,  6358,    13,   314,  1101, 14235, 33318,
           13,   775,   821, 22978,   422,  2669,    11,   360,    13,
           34,  1539,   422,   262,  2635,  2097,    11,   810,  1992,
         3759,  1301,   318,  7530,   284,  1826,   351,  3394,  1992,
        14840,  8144,    13,   383,  3249,   318,  2938,   284,  1011,
         1295,   287,   262, 25165,  4452,   286,   262,  3394,  1893,
           11,   475,   340,   338,   407,  1598,  1771,   262,  3249,
          481]], dtype=int32)>

In [152]:
tokenizer.decode(output[0], skip_special_tokens=True)

"This is a rush transcript. Copy may not be in its final form.\n\nAMY GOODMAN: This is Democracy Now!, democracynow.org, The War and Peace Report. I'm Amy Goodman. We're broadcasting from Washington, D.C., from the White House, where President Donald Trump is scheduled to meet with Russian President Vladimir Putin. The meeting is expected to take place in the Oval Office of the Russian president, but it's not clear whether the meeting will"

In [61]:
# max_input_length = 512
# max_target_length = 30


# def preprocess_function(examples):
#     model_inputs = tokenizer(
#         examples["review_body"], max_length=max_input_length, truncation=True
#     )
#     # Set up the tokenizer for targets
#     with tokenizer.as_target_tokenizer():
#         labels = tokenizer(
#             examples["review_title"], max_length=max_target_length, truncation=True
#         )

#     model_inputs["labels"] = labels["input_ids"]
#     return model_inputs

In [29]:
tokenized_datasets = english_home.map(preprocess_function, batched=True)

  0%|          | 0/11 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [45]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 10958
    })
    validation: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 241
    })
    test: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 283
    })
})

In [ ]:
output = model.generate(input_ids, max_length = 300, num_beams=5, no_repeat_ngram_size=2, early_stopping=True)